In [1]:
import numpy as np
import os
import torch

In [2]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data_dir, file_index):
        self.data_dir = data_dir
        self.file_index = file_index
        y = np.load(self.data_dir + "Y/" + str(self.file_index) + ".npy", allow_pickle=True)
        self.y = torch.from_numpy(y)
    
    def __getitem__(self, index):
        x = np.load(self.data_dir + "X/" + str(self.file_index) + ".npy", allow_pickle=True, mmap_mode='r')
        x = torch.from_numpy(x)
        return [x[index], self.y[index]]
    
    def __len__(self):
        return len(self.y)

In [3]:
data_set = MyDataset("/home/hansika/gem5/gem5/scripts/numpy_data/64_nodes/",0)

In [5]:
len(data_set)

399168

In [4]:
# samll set of data for trial run
data_set,data_set_1 = torch.utils.data.random_split(data_set, [100, 399068]) 

In [5]:
data_set[10]

<ipython-input-2-858f4186a916>:12: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  x = torch.from_numpy(x)


[tensor([[ 81,   1,   1,   1,   2,  51, 268,  17, 187,   1,   1,   1,   2, 132,
          100,  57,  41,  23,  50,   1,   1,   1,   2, 190,  49,   1,   1,   1,
            2, 234, 177,   1,   1,   1,   2, 358,  66, 152,   1,   1,   1,   2,
          146,   1,   1,   1,   2,  37, 154, 216,   1,   1,   1,   2,   9, 191,
            1,   1,   1,   2,  38,  30,   1,   1,   1,   2, 177,  38,  15,   1,
            1,   1,   2,   8,  44,   1,   1,   1,   2, 211,  45, 182, 317,  23,
           19,  70,   1,   1,   1,   2,  55, 231,  20, 206, 234,  62,  14,   2,
          128, 355, 140,   1,   1,   1,   1,   1,  89, 107, 134, 117, 123, 180,
           59, 161, 199,  41,   1,   1,   1,   2,  77, 205,  16,  20,  64,  78,
          210,  51,  58,  21, 512,  32,  21,   5,   3,   1,   1,   1,   2,  37,
          289,   1,   1,   1,   2, 105,  23,   7, 303,   1,   1,   1,   2, 218,
          253,  14, 132,  32, 129,   1,  25,   1,   1,   1,   2, 150,   3, 177,
           18,  19,   1,   1,   1,   2, 

In [9]:
# load full dataset from multiple files

list_of_dataset = []
number_of_files = 41
from torch.utils.data import ConcatDataset

for i in range(number_of_files):
    list_of_dataset.append(MyDataset("/home/hansika/gem5/gem5/scripts/numpy_data/64_nodes/",i))

full_dataset = ConcatDataset(list_of_dataset)

In [10]:
full_dataset[1]

[tensor([[  7,  50,  52,   1,   1,   1,   2,  31,  94,  60,   1,   1,   1,   2,
            4,   1,   1,   1,   2, 176, 156,  29,  44, 190,  21,   1,   1,   1,
            2,  10,   1,   1,   1,   2,  58,  57,  35,  24,   1,   1,   1,   2,
           27,   1,   1,   1,   2,  69,   1,   1,   1,   2,   3,   1,   1,   1,
            2,  22,  42,   1,   1,   1,   2,  68,  77,   8,   1,   1,   1,   2,
          172,   1,   1,   1,   2,  60,   1,   1,   1,   2,  35,   1,   1,   1,
            2,  19,   1,   1,   1,   2, 140,  78, 151, 333, 102,   1,   1,   1,
            1,   1,  66,  22,   1,   1,   1,   2,  38,  34, 123,   1,   1,   1,
            2,  26, 196,  76,   1,   1,   1,   2,   1,   1,   1,   1,   1,  68,
            1,   1,   1,   2,  54,   1,   1,   1,   2,  80,   1,   1,   1,   2,
           48,   1,   1,   1,   2,   1,   7, 127,  64,   1,   1,   1,   2, 134,
            1,   1,   1,   2,  46,   1,   1,   1,   2,  34, 140,   1,   1,   1,
            2,  19,  92,   3, 127,  65, 

In [5]:
len(full_dataset)

16257024

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# W1, W2, K1, K2 are hyper parameters that eventually needed training
W1 = 30
W2 = 10
K1= 2000
K2 = 1000

#dummy data to try the NN ( 2 arrays of size 450)
# dummy = torch.randn(2,450).view(-1,1,2,450)

# represents the whole CNN
class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, K1, (2,W1), stride=(2, 1))
        self.pool1 = nn.MaxPool2d((1, 5), stride=(1, 1))
        self.conv2 = nn.Conv2d(K1, K2, (1,W2), stride=(1, 1))
        self.pool2 = nn.MaxPool2d((1, 5), stride=(1, 1))
        
        self.fc1 = nn.Linear(1000*404, 3000) # need to automate arriving at this number (1000*254)
        self.fc2 = nn.Linear(3000, 800) 
        self.fc3 = nn.Linear(800,100)
        self.fc4 = nn.Linear(100,1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(-1, 1000*404)    
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return torch.sigmoid(x)
    
# ------------------- Training the CNN ------------------------------------- ##
# For now this code is only to show the structure, I need to add data preparation and modify code accordingly.

net = Net()

isTraining = True
if isTraining:
   
    BATCH_SIZE = 10
    EPOCHS = 1
    
    trainset = torch.utils.data.DataLoader(data_set, batch_size=BATCH_SIZE, shuffle=True)

    # learning rate of the adam optimizer should be a hyperparameter
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(EPOCHS):
        for data in trainset:
            X, y = data 
            net.zero_grad()  
            X = X.type(torch.FloatTensor)
            output = net(X.view(-1,1,2,450))
            loss = nn.CrossEntropyLoss()
            loss = loss(output, y)
            loss.backward() 
            optimizer.step()  
        print(loss)  


    correct = 0
    total = 0

    with torch.no_grad():
        for data in testset:
            X, y = data
            output = net(X.view(-1,784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total += 1

    print("Accuracy: ", round(correct/total, 3))    

/home/hansika/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
